In [19]:
import numpy as np
import os
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [11]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers as well as reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [12]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)

In [13]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [14]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [60]:
# load the network weights from best checkpoint
filename = "weights-improvement-20-1.9537.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [58]:
def sample(preds, temp=1.0):
    '''
    Helper function for sampling from softmax with different temperatures.
    
    Inputs:
    preds: output of softmax function
    temp: temperature to scale by
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [61]:
# generate emissions

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

temps = [0.01, 0.1, 0.2, 0.5, 1.0]
    
for temp in temps:    
    print('Temperature', temp)
    
    # generate characters
    newlines = 0
    while newlines < 14:
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)[0]

        # sample according to temperature
        idx = sample(prediction, temp)

        #index = np.argmax(prediction[0])
        result = int_to_char[idx]
        pattern.append(idx)
        pattern = pattern[1:len(pattern)]
        
        if result == '\n':
            newlines += 1
        
        # output result 
        sys.stdout.write(result)
        sys.stdout.flush()
    print()

Seed:
" ans more blessed than my barren rhyme?
n "
Temperature 0.01
o then whech wht ao allure a daarty 
  b b g qhree dore teen to lo thee iene thene,
the sore dore thet hasd the wiil hese soee,
  for thy sere loae that thet i soele tone,
  and tiae in then whth thes sooe rile dane,
 
to that i soan i then i which the whle,
  thene iane the wirle thy sore antheu hane,
  then tor tore toue, and then the toee hore thee,
  then thet iore cianke of tort sht tiee,
  thene ir thes sere, and thee in the worth,
  and thee ior sore an iath that whil sheee,
toon the seaes of the seaes sereed toee 


Temperature 0.1
 tha toeed i do whet i whtl the teme wiuh shae,
  thene ale there brens whth that they seee thee sire,
and that io then i would thy leve toou see,
  then toru the tere whth thes   shel fane toel in oh thee, the lone, ardee
the saafo oo hose socesue
toene to thee in the wirl of toettene,
and toenker fare the wirl hes st pose thele
then thou sool siat the world if thees then seed

  se